# Extraction - Subgraphs

This notebook contains all subgraph analyses referred to in chapter 4 of the thesis

It explores the graphs extracted from the 4 sections we consider in this project:
Related Terms, Derived Terms, Descendants and Etymology sections.


The following questions are answered:

* how many nodes and relations are contained in each section subgraph?
* how big is the redundancy within a section type?
* how does transitive reduction affect the graph?
* what are characteristics of the graph?
* what is the difference between the baseline extraction for the descendants and the etymology sections?
* what is the overlap between the subgraphs?
* what are the characteristics of the combined graph?

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("MongoEntryStore").setLevel(logging.INFO)
logging.getLogger("matplotlib").setLevel(logging.INFO)

import re
from collections import defaultdict, Counter
import json
import pickle
from pathlib import Path
import csv

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import wikitextparser as wtp
from tqdm import tqdm


import etymmap.specific_en

etymmap.specific_en.configure()

from etymmap.specific_en import consts

from etymmap.wiktionary import Wiktionary, MongoEntryStore

import etymmap.extraction
from etymmap.extraction.state import NoFuzzyGlossMatcher, State
from etymmap.utils import nonoverlapping
from etymmap.extraction.rules import to_sequence
from etymmap.extraction import (
    RelatedTermsExtractor,
    DerivedTermsExtractor,
    BaselineExtractor,
    DescendantsSectionExtractor,
    EtymologySectionExtractor,
)

from utils.subgraph_analysis import *

DATA_PATH = Path("./data/enwiktionary-20220601-pages-meta-current")
mongo_config = {
    "address": "mongodb://localhost:27017",
    "dbname": "enwiktionary",
    "collection": "20220601",
}


enw = Wiktionary(MongoEntryStore.from_config(mongo_config), default_namespaces=(0, 118))
etymmap.extraction.init_and_configure(
    enw, gloss_matcher=NoFuzzyGlossMatcher(), cache=DATA_PATH / "lexicon.pickle"
)

# Related terms

In [ ]:
# Related Terms, only Template and Link Extraction
related_relations = apply_extractor(
    RelatedTermsExtractor(), progress=True
).relation_store
related_graph = related_relations.finalize()
nx.write_gpickle(related_graph, DATA_PATH / "related.graph")
# Reduction types
related_relations.reduction_listener.counts()

In [ ]:
# redundant edges / all edges
redundancy(related_graph, related_relations.reduction_listener)

In [ ]:
# if already calculated: 
related_graph = nx.read_gpickle(DATA_PATH / "related.graph")

In [ ]:
# Basic stats
stats, csizes, components = describe(related_graph)
stats

In [ ]:
# number of componenents
len(csizes)

In [ ]:
# component size -> number of nodes in component(s) of this size
show_component_sizes(csizes)

In [ ]:
# Biggest component
show_languages_in_component(components[0])

In [ ]:
# second biggest
show_languages_in_component(components[1])

In [ ]:
# third biggest
show_languages_in_component(components[2])

In [ ]:
# contribution of biggest components
s = pd.Series(csizes).sort_values(ascending=False)
df = pd.concat([s, s / s.sum()], axis=1).head(10)
df.columns = ["component size", "ratio of graph"]

In [ ]:
# most common languages
related_langs = language_counts(related_graph)
related_langs.head(25)

In [ ]:
# in-out-degrees
related_degrees, related_degree_grid = in_out_degree_grid(related_graph)

In [ ]:
# undirected degrees, as there are no (semantically) directed relations
s = pd.Series(related_degrees)
undirected = pd.DataFrame.from_records(s).sum(axis=1)
undirected.index = s.index
undirected.sort_values(ascending=False).head(25)

In [ ]:
# which ratio of nodes are branching?
(undirected > 2).sum() / len(undirected)

# Derived Relations

In [ ]:
# Derived Relations, only Template and Link extraction
derived_relations = apply_extractor(
    DerivedTermsExtractor(), progress=True
).relation_store
derived_graph = derived_relations.finalize()
nx.write_gpickle(derived_graph, DATA_PATH / "derived.graph")
# Reduction types
derived_relations.reduction_listener.counts()

In [ ]:
# redundant edges / all edges
redundancy(derived_graph, derived_relations.reduction_listener)

In [ ]:
# if already calculated:
# derived_graph = nx.read_gpickle(DATA_PATH / "derived.graph")

In [ ]:
# Basic stats
stats, csize, components = describe(derived_graph)
stats

In [ ]:
# component size -> number of nodes in component(s) of this size
show_component_sizes(csize)

In [ ]:
# Languages in biggest component
show_languages_in_component(components[0])

In [ ]:
# Second biggest component
show_languages_in_component(components[1])

In [ ]:
# contribution of biggest components
print([len(components[i]) for i in range(3)])
print([len(components[i]) / len(derived_graph.nodes) for i in range(3)])

In [ ]:
# most common languages in derived terms
derived_langs = language_counts(derived_graph)
derived_langs.head(25)

In [ ]:
# undirected degrees, as there are no (semantically) directed relations
derived_degrees, grid = in_out_degree_grid(derived_graph)
s = pd.Series(derived_degrees)
undirected = pd.DataFrame.from_records(s).sum(axis=1)
undirected.index = s.index
undirected.sort_values(ascending=False).head(25)

In [ ]:
# which ratio of nodes are branching?
(undirected > 2).sum() / len(undirected)

# Descendants Baseline

In [ ]:
# Descendants, only Template extraction
descendants_base_extractor = BaselineExtractor(consts.DESCENDANTS)
descendants_base_relations = apply_extractor(
    descendants_base_extractor, progress=True
).relation_store
print(descendants_base_relations.reduction_listener.counts())
descendants_base_graph = descendants_base_relations.finalize()
nx.write_gpickle(descendants_base_graph, DATA_PATH / "descendants_base.graph")
# Reduction types
descendants_base_relations.reduction_listener.counts()

In [ ]:
redundancy(descendants_base_graph, descendants_base_relations.reduction_listener)

In [ ]:
# if already calculated:
# descendants_base_graph = nx.read_gpickle(DATA_PATH / "descendants_base.graph")

In [ ]:
stats, csize, comps = describe(descendants_base_graph)

In [ ]:
stats

In [ ]:
show_component_sizes(csize)

In [ ]:
language_counts(descendants_base_graph).head(25)

# Descendants (nested)

In [ ]:
# Descendants, with analysis of nested item structure and adjust of edge direction
descendants_relations = apply_extractor(DescendantsSectionExtractor(), progress=True, swap_historic_languages=True).relation_store
print(descendants_relations.reduction_listener.counts())
descendants_graph = descendants_relations.finalize()
nx.write_gpickle(descendants_graph, DATA_PATH / "descendants.graph")
descendants_relations.reduction_listener.counts()

In [ ]:
# which relation types cause conflicts?
rl = descendants_relations.reduction_listener
rl.events[rl.Event.INCOMPATIBLE]

In [ ]:
redundancy(descendants_graph, descendants_relations.reduction_listener)

In [ ]:
# if already calculated:
# descendants_graph = nx.read_gpickle(DATA_PATH / "descendants.graph")

In [ ]:
stats, csizes, comps = describe(descendants_graph)

In [ ]:
stats

In [ ]:
show_component_sizes(csizes)

In [ ]:
# biggest component
show_languages_in_component(comps[0])

In [ ]:
# second biggest component
show_languages_in_component(comps[1])

# Compare Descendants Baseline with improved

In [ ]:
# load precomputed graphs
descendants_base_graph = nx.read_gpickle(DATA_PATH / "descendants_base.graph")
descendants_graph = nx.read_gpickle(DATA_PATH / "descendants.graph")

In [ ]:
# Overlap
df, nodes = get_overlap(descendants_base_graph, descendants_graph)
df

## Compare the node degrees

In [ ]:
degrees_base, in_out_grid_base = in_out_degree_grid(descendants_base_graph)
degrees_nested, in_out_grid_nested = in_out_degree_grid(descendants_graph)

In [ ]:
# track most common node degree changes
common_nodes = set(degrees_base).intersection(degrees_nested)
degrees_change = Counter()
for node in common_nodes:
    base = degrees_base[node]
    nested = degrees_nested[node]
    if base != nested:
        degrees_change[(base, nested)] += 1
degrees_change.most_common(n=10)

In [ ]:
# nodes that did not have successors in base but have successors in improved
sum(
    count
    for ((base_in, base_out), (nested_in, nested_out)), count in degrees_change.items()
    if base_out == 0 and nested_out != 0
)

In [ ]:
# compare shortest path statistics: bigger values should correspond to longer chains
desc_sp_base = shortest_paths(descendants_base_graph)
desc_sp = shortest_paths(descendants_graph)
desc_sp.mean(), desc_sp_base.mean()

In [ ]:
# ratio
desc_sp["avg_shortest_path"].mean() / desc_sp_base["avg_shortest_path"].mean()

In [ ]:
# for how many nodes did the value grow / shrink?
diff = desc_sp.set_index("node") - desc_sp_base.set_index("node")
(diff["avg_shortest_path"] < 0).sum(), (diff["avg_shortest_path"] > 0).sum()

# Etymology Baseline

In [ ]:
# Etymology, only Template extraction
etymology_base_extractor = BaselineExtractor(consts.ETYMOLOGY_SECTION)
etymology_base_relations = apply_extractor(etymology_base_extractor, progress=True).relation_store
print(etymology_base_relations.reduction_listener.counts())
etymology_base_graph = etymology_base_relations.finalize()
nx.write_gpickle(etymology_base_graph, DATA_PATH / "etymology_base.graph")
etymology_base_relations.reduction_listener.counts()

In [ ]:
redundancy(graph, etymology_base_relations.reduction_listener)

In [ ]:
# if already computed
# etymology_base_graph = nx.read_gpickle(DATA_PATH / "etymology_base.graph")

In [ ]:
stats, csizes, comps = describe(etymology_base_graph)

In [ ]:
stats

In [ ]:
sorted(csizes, reverse=True)[:10]

# Etymology Extractor (+)

In [ ]:
# Etymology, additionally with chain resolution and rule mechanism
etymology_extractor = EtymologySectionExtractor(chain_resolution=True)
etymology_relations = apply_extractor(etymology_extractor, progress=True, swap_historic_languages=True)

In [ ]:
# how often could chain resolution be applied (or not)
chain_links, containing_lexeme_links = etymology_extractor.chain_resolution_counter
chain_links, containing_lexeme_links, chain_links / (chain_links + containing_lexeme_links)

In [ ]:
# how often was each rule applied (before reduction)
etymology_extractor.rules.counts

In [ ]:
# Reduction and Redundancy
print(etymology_relations.relation_store.reduction_listener.counts())
redundancy(graph, etymology_relations.relation_store.reduction_listener)

In [ ]:
# final graph
etymology_plus = etymology_relations.relation_store.finalize(rm_cycles_inplace=True)
etymology_relations.relation_store.reduction_listener.counts()

In [ ]:
nx.write_gpickle(graph, DATA_PATH / "etymology.graph")

In [ ]:
# if already computed
# etymology_plus = nx.read_gpickle(DATA_PATH / "etymology.graph")

In [ ]:
stats, csizes, comps = describe(etymology_plus)

In [ ]:
stats

In [ ]:
# component size -> number of nodes in components of this size
show_component_sizes(csizes, bins=np.logspace(0, 6.5, 25))

In [ ]:
# biggest component
show_languages_in_component(comps[0])

In [ ]:
# second biggest component
show_languages_in_component(comps[1])

In [ ]:
# biggest components
sorted(csizes, reverse=True)[:5]

In [ ]:
# number of small components
sum(1 for s in csizes if s < 10)

# Compare Etymology Baseline and Etymology (+)

In [ ]:
# Load precomputed graphs
etymology_base = nx.read_gpickle(DATA_PATH / "etymology_base.graph")
etymology_plus = nx.read_gpickle(DATA_PATH / "etymology.graph")

In [ ]:
# Intersections of subgraphs
overlap, examples = get_overlap(etymology_base, etymology_plus)
overlap

In [ ]:
# Compare the types: difference plus to baseline
base_types = pd.Series(Counter(e[2].name for e in etymology_base.edges))
plus_types = pd.Series(Counter(e[2].name for e in etymology_plus.edges))
plus_types.sub(base_types, fill_value=0).astype(int).sort_values(ascending=False)

In [ ]:
# Average shortest paths (sampled)
etym_shortest_paths_base = shortest_paths(etymology_base, frac=0.01)
etym_shortest_paths_plus = shortest_paths(etymology_plus, frac=0.01)
etym_shortest_paths_base.mean(), etym_shortest_paths_plus.mean()

In [ ]:
# degrees
degrees_base, in_out_grid_base = in_out_degree_grid(etymology_base)
degrees_plus, in_out_grid_plus = in_out_degree_grid(etymology_plus)

In [ ]:
# most common degree changes
common_nodes = set(degrees_base).intersection(degrees_plus)
degrees_change = Counter()
for node in common_nodes:
    base = degrees_base[node]
    plus = degrees_plus[node]
    if base != plus:
        degrees_change[(base, plus)] += 1
degrees_change.most_common()

In [ ]:
# degree changes in the grid
in_out_compare = (in_out_grid_plus - in_out_grid_base).fillna(0).astype(int)
in_out_compare.sort_index().iloc[:10, :10]

## Etymology and Descendants

In [ ]:
# Etymology and Descendants

etymology_plus = nx.read_gpickle("etymology.graph")
descendants_graph = nx.read_gpickle(DATA_PATH / "descendants.graph")

In [ ]:
overlap, examples = get_overlap(etymology_plus, descendants_graph)
overlap

In [ ]:
# ratio of exclusive descendants nodes  / edges
overlap.loc["only_right", "nodes"] / descendants_graph.number_of_nodes(), \
overlap.loc["only_right", "edges"] / descendants_graph.number_of_edges()

# Combine all extractors 

In [ ]:
all_relations = apply_extractor(
    RelatedTermsExtractor(),
    DerivedTermsExtractor(),
    DescendantsSectionExtractor(),
    EtymologySectionExtractor(),
    swap_historic_languages=True,
    progress=True,
).relation_store

In [ ]:
print(all_relations.reduction_listener.counts())
graph = all_relations.finalize()
nx.write_gpickle(graph, DATA_PATH / "complete.graph")
all_relations.reduction_listener.counts()

In [ ]:
redundancy(graph, all_relations.reduction_listener)

In [ ]:
# if already computed
# graph = nx.read_gpickle(DATA_PATH / "complete.graph")

In [ ]:
stats, csizes, comps = describe(graph)

In [ ]:
stats

In [ ]:
# ratio of biggest components
[s / graph.number_of_nodes() for s in sorted(csizes, reverse=True)][:10]

In [ ]:
# Count node types
node_types = pd.Series(Counter(type(n) for n in graph.nodes))
node_types

In [ ]:
# node types, relative
node_types / node_types.sum()

In [ ]:
# How much of the lexicon is part of the graph?
def lexical_nodes():
    for by_term in State.lexicon.single_meanings.values():
        if isinstance(by_term, list):
            yield from by_term
        else:
            yield by_term
    for by_term in State.lexicon.multi_meanings.values():
        for by_lang in by_term.values():
            yield from by_lang

all_nodes = {node.id for node in graph.nodes}
contained = not_contained = 0
not_contained_examples = set()
for node in lexical_nodes():
    if node.id in all_nodes:
        contained += 1
    else:
        not_contained += 1
        if len(not_contained_examples) < 1000:
            not_contained_examples.add(node)
all_ = contained + not_contained
contained, contained/all_, not_contained, all_

In [ ]:
# That's not very much, how many entries do at all contain relevant sections?
with_any_section = \
set((entry["title"], entry["language"]) for *_, entry in enw.sections(consts.ALL_ETYMOLOGY_SECTIONS))
len(with_any_section)

In [ ]:
# Most common languages
languages_in_graph = language_counts(graph)
languages_in_graph.head(20)

In [ ]:
# degrees
degrees, grid = in_out_degree_grid(graph)
grid.loc[:10, :10]

In [ ]:
# ratio of in-degrees
(grid.sum(axis=1) / graph.number_of_nodes()).head(10)

In [ ]:
# ratio of out-degrees
(grid.sum() / graph.number_of_nodes()).head(10)

In [ ]:
# as frame
degrees = pd.Series(degrees)
degrees_df = pd.DataFrame.from_records(degrees.values, columns=["in", "out"])
degrees_df.index = degrees.index
degrees_df.mean()

In [ ]:
# highest in-degree nodes
degrees_df.sort_values("in", ascending=False).head(25)

In [ ]:
# highest out-degree nodes
degrees_df.sort_values("out", ascending=False).head(25)

## Page Rank

In [ ]:
biggest_component = graph.subgraph(comps[0])
# make undirected, then calculate page rank
evc = nx.pagerank(nx.Graph(biggest_component), max_iter=1000)
page_rank = pd.Series(evc).sort_values(ascending=False)
page_rank.head(30)

## Communities

In [ ]:
# Communities and between connections

from networkx.algorithms.community import louvain_communities
comms = louvain_communities(comp_graph, threshold=10**-6, resolution=0.0001)
comm_sizes = pd.Series([len(c) for c in comms])
comm_sizes.hist(bins=range(50))

In [ ]:
# select biggest communities and find nodes on which they have intra-community-connections
biggest = [comms[i] for i in comm_sizes.sort_values(ascending=False).head(50).index]
connecting_nodes = Counter()

combinations = tqdm.tqdm(((sg, sg2) for i, sg in enumerate(biggest) for sg2 in biggest[i+1:]), total=sum(range(len(biggest))))
for sg, sg2 in combinations:
    for n in sg:
        for n2 in sg2:
            if graph.has_edge(n, n2) or graph.has_edge(n2, n):
                connecting_nodes[n] += 1
                connecting_nodes[n2] += 1
pd.Series(connecting_nodes).sort_values(ascending=False).head(10)

## Nodes that connect many languages

In [ ]:
# could use bfs, but don't want to copy the graph to make it undirected
def nodes_in_radius2(graph, node):
    s, s2 = set(), set()
    s.update(graph.successors(node))
    s.update(graph.predecessors(node))
    for n in s:
        s2.update(graph.successors(n))
        s2.update(graph.predecessors(n))
    return s | s2

langs_in_radius2 = defaultdict(Counter)

nodes = tqdm.tqdm((node for node in graph if graph.degree(node) >= 10), total=107000)
for node in nodes:
    for node2 in nodes_in_radius2(graph, node):
        if isinstance(node2, LexemeBase):
            langs_in_radius2[node][node2.language] += 1
            
langs_in_radius2 = {k: v for k, v in langs_in_radius2.items() if len(v) > 10}

In [ ]:
len(langs_in_radius2)

In [ ]:
# nodes that connect the most languages
{k: len(langs_in_radius2[k]) for k in sorted(langs_in_radius2, key=lambda k: len(langs_in_radius2[k]), reverse=True)[:100]}